In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
## Load the model scaler pickle and one hot encoder
model = load_model("model.h5")

## Load the encoder and scaler
with open("onehot_encoder_geo.pkl", "rb") as file:
    onehot_encoder_geo = pickle.load(file=file)

with open("label_encoder_gender.pkl", "rb") as file:
    label_encoder_gender = pickle.load(file=file)

with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file=file)

In [32]:
## Example Input Data
input_data = {
    "CreditScore": 300,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 5000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000
}


In [33]:
input_df = pd.DataFrame([input_data])

In [34]:
## Onehot encode "Geography"

geo_encoded = onehot_encoder_geo.transform([[input_data["Geography"]]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))

d:\Data Science Bootcamp\Deep Learning\1. ann classification\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [35]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [36]:
## Encode categorical variables

input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])

In [37]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,300,France,1,40,3,5000,2,1,1,50000


In [38]:
## concatenation one hot encoded

input_df = pd.concat([input_df.drop("Geography", axis=1),geo_encoded_df], axis=1)

In [39]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,300,1,40,3,5000,2,1,1,50000,1.0,0.0,0.0


In [40]:
## Scaling the input data

input_scaled = scaler.transform(input_df)
input_scaled

array([[-3.64930447,  0.91324755,  0.10479359, -0.69539349, -1.13841561,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [41]:
## prediction churn

prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


array([[0.03035239]], dtype=float32)

In [42]:
prediction_probablity = prediction[0][0]
prediction_probablity

np.float32(0.030352393)

In [43]:
if prediction_probablity > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
